In [1]:
import json
from typing import Dict, Any

from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from core.powerfulchain import PowerfulAPIChain

llm_kwargs: Dict[str, Any] = {
    "model": "llama3:instruct", # Model name
    "temperature": 0,
    "num_ctx": 2048, # Context size
}

llm = Ollama(**llm_kwargs)

In [2]:
# Example of processed API docs from https://api.xcloud.coralio.fr/openapi.json

_api_docs = {'base_url': 'https://api.xcloud.coralio.fr/',
 'endpoints': [
        {'/api/chat': {'post': {'tags': ['Chat'
                    ],
     'summary': 'Create Chat',
     'description': 'Creates a chat',
     'operationId': 'create_chat_api_chat_post',
     'requestBody': {'required': True,
      'content': {'application/json': {'schema': {'$ref': {'properties': {'user_id': {'anyOf': [
                                                    {'type': 'string'
                                                    },
                                                    {'type': 'string', 'format': 'uuid'
                                                    }
                                                ],
            'title': 'User Id'
                                            },
           'type': {'type': 'string', 'enum': ['dm', 'gc'
                                                ], 'title': 'Type'
                                            },
           'name': {'anyOf': [
                                                    {'type': 'string'
                                                    },
                                                    {'type': 'null'
                                                    }
                                                ],
            'title': 'Name',
            'default': ''
                                            },
           'participants': {'items': {'anyOf': [
                                                        {'type': 'string'
                                                        },
                                                        {'type': 'string', 'format': 'uuid'
                                                        }
                                                    ]
                                                },
            'type': 'array',
            'title': 'Participants'
                                            }
                                        },
          'type': 'object',
          'required': ['user_id', 'type', 'participants'
                                        ],
          'title': 'ChatCreate'
                                    }
                                }
                            }
                        }
                    },
     'responses': {'201': {'description': 'Successful Response',
       'content': {'application/json': {'schema': {}
                                }
                            }
                        },
      '422': {'description': 'Validation Error',
       'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'
                                    }
                                }
                            }
                        }
                    },
     'apiUrl': 'https://api.xcloud.coralio.fr/api/chat'},
    'get': {'tags': ['Chat'
                        ],
     'summary': 'List Chats',
     'operationId': 'list_chats_api_chat_get',
     'parameters': [
                            {'name': 'user_id',
       'in': 'query',
       'required': True,
       'schema': {'type': 'string', 'title': 'User Id'
                                }
                            },
                            {'name': 'q',
       'in': 'query',
       'required': False,
       'schema': {'type': 'string', 'default': '', 'title': 'Q'
                                }
                            },
                            {'name': 'type',
       'in': 'query',
       'required': False,
       'schema': {'enum': ['dm', 'gc'
                                    ], 'type': 'string', 'title': 'Type'
                                }
                            },
                            {'name': 'limit',
       'in': 'query',
       'required': False,
       'schema': {'type': 'integer',
        'maximum': 25,
        'minimum': 1,
        'default': 10,
        'title': 'Limit'
                                }
                            },
                            {'name': 'offset',
       'in': 'query',
       'required': False,
       'schema': {'type': 'integer',
        'minimum': 0,
        'default': 0,
        'title': 'Offset'
                                }
                            }
                        ],
     'responses': {'200': {'description': 'Successful Response',
       'content': {'application/json': {'schema': {}
                                    }
                                }
                            },
      '422': {'description': 'Validation Error',
       'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'
                                        }
                                    }
                                }
                            }
                        },
     'apiUrl': 'https://api.xcloud.coralio.fr/api/chat'}}}]}
api_docs = json.dumps(_api_docs, indent=2)

In [3]:
api_url_template = """
Given the following API Documentation for XCloud's official 
FinOps API: {api_docs}
Your task is to construct the most efficient API URL to answer 
the user's question, ensuring the 
call is optimized to include only necessary information.
Only return the API URL and nothing else.
If a parameter is not required and does not have a
default value, do NOT include it in the API URL, unless you think
it's relevant to the user's question.
You should also extract the request METHOD and generate the BODY data in JSON format
according to the user question if necessary.
The parameters' names and BODY data keys MUST be obtained from the provided context.
Do NOT make up parameters' names.
The BODY data can be an empty JSON.
Output the API URL, METHOD and BODY. Join them with `|`. DO NOT GIVE ANY EXPLANATION.

Question: {question}

Output:
"""
api_url_prompt = PromptTemplate(input_variables=['api_docs', 'question'],
                                template=api_url_template)

api_response_template = """"
With the API Documentation for XCloud's official API: {api_docs} 
and the specific user question: {question} in mind,
and given this API URL: {api_url} for querying, here is the 
response from XCloud's API: {api_response}. 
Please respond to the user's question, 
omitting technical details like response format, and 
focusing on delivering the answer with clarity and conciseness.
Response:
"""
api_response_prompt = PromptTemplate(input_variables=['api_docs', 
                                                      'question', 
                                                      'api_url',
                                                      'api_response'],
                                     template=api_response_template)

In [4]:
# Instantiate the new PowerfulAPIChain subclass

chain = PowerfulAPIChain.from_llm_and_api_docs(
    llm,
    api_docs=api_docs,
    headers=None,
    api_url_prompt=api_url_prompt,
    api_response_prompt=api_response_prompt,
    verbose=True,
    limit_to_domains=["https://api.xcloud.coralio.fr/"],
)

c:\Users\MSI\Desktop\cor\imx\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
# Example user request that will prompt the LLM to make an API POST request

participants = ['0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1',
                'ff0d95a2-69ca-41ca-9c91-064259adb7c4']
user_input = (
    f"""Create a chat with the user being ff0d95a2-69ca-41ca-9c91-064259adb7c4 and
    the participants being {participants}, the type is 'gc', the name is 'my group chat 5'"""
)
response = await chain.ainvoke(user_input)



> Entering new PowerfulAPIChain chain...
Request info: https://api.xcloud.coralio.fr/api/chat|POST|{"user_id":"ff0d95a2-69ca-41ca-9c91-064259adb7c4","type":"gc","name":"my group chat 5","participants":["0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1","ff0d95a2-69ca-41ca-9c91-064259adb7c4"]}
API URL: https://api.xcloud.coralio.fr/api/chat

                    Request method: POST

                    Request body: {"user_id":"ff0d95a2-69ca-41ca-9c91-064259adb7c4","type":"gc","name":"my group chat 5","participants":["0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1","ff0d95a2-69ca-41ca-9c91-064259adb7c4"]}

> Finished chain.


In [6]:
print(
    f"Human: {response['question']}\n"
    f"Assistant: {response['output']}"
)

Human: Create a chat with the user being ff0d95a2-69ca-41ca-9c91-064259adb7c4 and
    the participants being ['0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1', 'ff0d95a2-69ca-41ca-9c91-064259adb7c4'], the type is 'gc', the name is 'my group chat 5'
Assistant: Your chat has been successfully created! The chat ID is `882965b8-6c41-4c74-ba8f-af7f7f151b91`. It's a group chat of type `gc` with the name `my group chat 5`. You are the creator and one of the participants. The other participant is `Raed Chebbi` (username: `r.chebbi@coral-io.fr`).


In [7]:
# Verify that the request was completed successfully
import requests
# TODO